In [1]:
# arxiv-Research
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
# we used the inbulit tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name


'wikipedia'

In [3]:
# using inbulit arxiv tool
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools=[wiki,arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\RAG-Doc-Q&A\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

In [6]:
# creating custom tools[RAG Tool]
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

from langchain_text_splitters import RecursiveCharacterTextSplitter



In [7]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings=(
  OllamaEmbeddings(model="gemma2:2b")
)

In [8]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000168C52FDFC0>)

In [9]:
# making the retriever as retriever tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","search any information about langsmith")
retriever_tool.name

'langsmith-search'

In [10]:
tools.append(retriever_tool)
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\RAG-Doc-Q&A\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000168BD38EA70>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000168C52FDFC0>), document_prompt

In [11]:
# run all this tools with agents with ll models
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")



In [12]:
# creating the prompt template
# hub a prompt templated already created and uploaded inn hub
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")



d:\RAG-Doc-Q&A\venv\lib\site-packages\langsmith\client.py:5431: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [13]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [15]:
# to run the we create a agent executer
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executer


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [23]:
# from langchain_experimental.llms.ollama_functions import OllamaFunctions

# model = OllamaFunctions(model="llama3", format="json")

In [16]:
agent_executer.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input['postfix'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, E

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.'}

In [17]:
agent_executer.invoke({"input":"tell me about llm models"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Large language model'}`


Page: Large language model
Summary: A large language model (LLM) is a computational model capable of language generation or other natural language processing tasks. As language models, LLMs acquire these abilities by learning statistical relationshipLarge language models (LLMs) are a type of artificial intelligence that excel at understanding and generating human language. 

They are trained on massive datasets of text and code, allowing them to learn patterns and relationships within language. This enables them to perform a wide range of tasks, including:

* **Text Generation:** Writing stories, articles, poems, and other types of creative content.
* **Translation:** Converting text from one language to another.
* **Summarization:** Condensing large amounts of text into shorter summaries.
* **Question Answering:** Providing answers to questions based on given information.
* **Code

{'input': 'tell me about llm models',
 'output': 'Large language models (LLMs) are a type of artificial intelligence that excel at understanding and generating human language. \n\nThey are trained on massive datasets of text and code, allowing them to learn patterns and relationships within language. This enables them to perform a wide range of tasks, including:\n\n* **Text Generation:** Writing stories, articles, poems, and other types of creative content.\n* **Translation:** Converting text from one language to another.\n* **Summarization:** Condensing large amounts of text into shorter summaries.\n* **Question Answering:** Providing answers to questions based on given information.\n* **Code Generation:** Writing computer code in various programming languages.\n\nLLMs have the potential to revolutionize many industries, from customer service and education to research and entertainment.\n\n\n\n'}

In [19]:
agent_executer.invoke({"input":"what the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c"Attention Is All You Need" is a groundbreaking paper in the field of natural language processing that introduces the Transformer architecture. 

This architecture relies entirely on attention mechanisms, dispensing with recurrent or convolutional networks altogether. The Transformer has revolutionized the field, leading to significant improvements in machine translation, text summarization, and other NLP tasks.  


> Finished chain.


{'input': 'what the paper 1706.03762 about?',
 'output': '"Attention Is All You Need" is a groundbreaking paper in the field of natural language processing that introduces the Transformer architecture. \n\nThis architecture relies entirely on attention mechanisms, dispensing with recurrent or convolutional networks altogether. The Transformer has revolutionized the field, leading to significant improvements in machine translation, text summarization, and other NLP tasks.  \n'}

In [20]:
agent_executer.invoke({"input":"give me the summary about what i asked"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'what is a summary about what i asked'}`


Page: What Is a Woman?
Summary: What Is a Woman? is a 2022 American online film about gender and transgender issues, directed by Justin Folk and presented by conservative political commentator Matt Walsh. The film was released by conservative website
Invoking: `wikipedia` with `{'query': 'what is a summary about gender and transgender issues'}`


Page: What Is a Woman?
Summary: What Is a Woman? is a 2022 American online film about gender and transgender issues, directed by Justin Folk and presented by conservative political commentator Matt Walsh. The film was released by conservative websiteGender and transgender issues are complex and multifaceted. 

At their core, gender and transgender identities relate to a person's internal sense of their own gender, which may or may not align with the sex they were assigned at birth. 

Transgender people have a gender identit

{'input': 'give me the summary about what i asked',
 'output': "Gender and transgender issues are complex and multifaceted. \n\nAt their core, gender and transgender identities relate to a person's internal sense of their own gender, which may or may not align with the sex they were assigned at birth. \n\nTransgender people have a gender identity that differs from the sex they were assigned at birth. This can involve a range of experiences, including:\n\n* **Transgender men:** Assigned female at birth but identify as male.\n* **Transgender women:** Assigned male at birth but identify as female.\n* **Non-binary:** Individuals who do not identify exclusively as male or female.\n\nUnderstanding gender and transgender issues requires recognizing that:\n\n* **Gender is a spectrum:** It's not simply male or female, and individuals may experience their gender in various ways.\n* **Gender identity is distinct from sexual orientation:** A person's gender identity does not determine who they are